In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import nest_asyncio


sys.path.insert(0, os.path.abspath('..'))
nest_asyncio.apply()

In [2]:
import logging


logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
# logging.getLogger('pylatexenc.latexwalker').setLevel(logging.ERROR)
# logging.getLogger('httpx').setLevel(logging.WARNING)
# logging.getLogger('openai').setLevel(logging.WARNING)

In [4]:
from math_rag.infrastructure.containers import InfrastructureContainer


RESET = True

# containers
infrastructure_container = InfrastructureContainer()
infrastructure_container.init_resources()
infrastructure_container.wire(modules=[__name__])

application_container = infrastructure_container.application_container()
application_container.init_resources()
application_container.wire(modules=[__name__])

# seed
for object_seeder in infrastructure_container.object_seeders():
    object_seeder.seed(reset=RESET)

for document_seeder in infrastructure_container.document_seeders():
    await document_seeder.seed(reset=RESET)

for embedding_seeder in infrastructure_container.embedding_seeders():
    await embedding_seeder.seed(reset=RESET)

# index
for document_indexer in infrastructure_container.document_indexers():
    await document_indexer.index(reset=RESET)

2025-06-13 23:30:55,234 - INFO - HTTP Request: GET http://qdrant:6333 "HTTP/1.1 200 OK"
2025-06-13 23:30:55,239 - INFO - HTTP Request: DELETE http://qdrant:6333/collections/mathexpressiondescriptionembedding "HTTP/1.1 200 OK"
2025-06-13 23:30:55,240 - INFO - HTTP Request: GET http://qdrant:6333/collections/mathexpressiondescriptionembedding/exists "HTTP/1.1 200 OK"
2025-06-13 23:30:55,299 - INFO - HTTP Request: PUT http://qdrant:6333/collections/mathexpressiondescriptionembedding "HTTP/1.1 200 OK"


In [ ]:
from uuid import UUID

from math_rag.core.enums import MathExpressionDatasetBuildStage
from math_rag.core.models import MathExpressionDataset


math_expression_dataset_builder_service = (
    application_container.math_expression_dataset_builder_service()
)
math_expression_dataset_repository = application_container.math_expression_dataset_repository()


current_dataset = MathExpressionDataset(
    # build_from_dataset_id=UUID('dd5b1e2d-dc0c-4ae9-9f24-2fd2d37f42fa'),
    # build_from_stage=MathExpressionDatasetBuildStage.LOAD_MATH_EXPRESSION_SAMPLES,
)
await math_expression_dataset_repository.insert_one(current_dataset)
await math_expression_dataset_builder_service.build(current_dataset)

In [2]:
from math_rag.application.models.assistants import MathExpressionLabelerAssistantOutput


MathExpressionLabelerAssistantOutput.model_json_schema()

{'$defs': {'MathExpressionLabelEnum': {'enum': ['equality',
    'inequality',
    'constant',
    'variable',
    'other'],
   'title': 'MathExpressionLabelEnum',
   'type': 'string'}},
 'properties': {'label': {'$ref': '#/$defs/MathExpressionLabelEnum'}},
 'required': ['label'],
 'title': 'MathExpressionLabelerAssistantOutput',
 'type': 'object'}

### Correct

In [6]:
incorrect_katex = r'd\omega = \theta \w \omega'
error = (
    r'KaTeX parse error: Undefined control sequence: \w at position 18: …omega = \theta \̲w̲ ̲\omega'
)

In [7]:
from math_rag.application.models.assistants import KatexCorrectorAssistantInput


input = KatexCorrectorAssistantInput(katex=incorrect_katex, error=error)

In [ ]:
katex_corrector_assistant = application_container.katex_corrector_assistant()

output = await katex_corrector_assistant.assist(input)
corrected_katex = output.katex
print(corrected_katex)
display(Math(corrected_katex))

### Dataset

#### Download

In [ ]:
from datasets import load_dataset
from datasets.download import DownloadConfig
from decouple import config


HF_USERNAME = config('HF_USERNAME', default=None)
HF_TOKEN = config('HF_TOKEN', default=None)

download_config = DownloadConfig(
    max_retries=3,
    disable_tqdm=True,
)

dataset_dict = load_dataset(
    path=f'{HF_USERNAME}/mathexpressiondataset',
    split=None,
    download_config=download_config,
    token=HF_TOKEN,
    trust_remote_code=True,
)

In [8]:
from typing import cast

from datasets import ClassLabel


class_label = cast(ClassLabel, dataset_dict['train'].features['label'])
class_label.names

['equality', 'inequality', 'constant', 'variable', 'other']

### OpenAI finish reasons

In [4]:
response_with_length_finish_reason = {
    'id': 'batch_req_68338415d224819095e7e42c4aeac8f8',
    'custom_id': 'b9b237e5-d1c8-4348-9c90-f29e998228a6',
    'response': {
        'status_code': 200,
        'request_id': 'a12fbd18fe2366571f083c2b6e707c96',
        'body': {
            'id': 'chatcmpl-BbB9Kx0P0WJlR2r4I1SuRkYa2Mvn0',
            'object': 'chat.completion',
            'created': 1748200694,
            'model': 'gpt-4.1-nano-2025-04-14',
            'choices': [
                {
                    'index': 0,
                    'message': {
                        'role': 'assistant',
                        'content': 'some too long content...',
                        'refusal': None,
                        'annotations': [],
                    },
                    'logprobs': None,
                    'finish_reason': 'length',
                }
            ],
            'usage': {
                'prompt_tokens': 285,
                'completion_tokens': 1024,
                'total_tokens': 1309,
                'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
                'completion_tokens_details': {
                    'reasoning_tokens': 0,
                    'audio_tokens': 0,
                    'accepted_prediction_tokens': 0,
                    'rejected_prediction_tokens': 0,
                },
            },
            'service_tier': 'default',
            'system_fingerprint': 'fp_eede8f0d45',
        },
    },
    'error': None,
}

In [ ]:
from openai import NOT_GIVEN
from openai.lib._parsing._completions import (
    parse_chat_completion,
)
from openai.types.chat import ChatCompletion


chat_completion = ChatCompletion(**response_with_length_finish_reason['response']['body'])

for choice in chat_completion.choices:
    if choice.finish_reason == 'length' or choice.finish_reason == 'content_filter':
        print(choice.finish_reason)
        pass

target = parse_chat_completion(
    response_format=NOT_GIVEN,
    input_tools=NOT_GIVEN,
    chat_completion=chat_completion,
)

length


LengthFinishReasonError: Could not parse response content as the length limit was reached - CompletionUsage(completion_tokens=1024, prompt_tokens=285, total_tokens=1309, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

In [8]:
from math_rag.core.enums.arxiv import MathCategory


print(str(MathCategory.AC.__class__))

<enum 'MathCategory'>


In [10]:
from math_rag.shared.utils import TypeUtil


TypeUtil.to_fqn(MathCategory.AC.__class__)

'math_rag.core.enums.arxiv.MathCategory'